# Compute performance metrics for the given Y and Y_score without sklearn

In [2]:
import numpy as np
import pandas as pd
# other than these two you should not import any other packages


## A. Compute performance metrics for the given data

In [3]:
df_a=pd.read_csv(r'C:\Users\HP\OneDrive\Applied ai\Module 3\assignment\Performance_metrics_\5_a.csv')
df_a.head()

,y,proba
0,1.0,0.637387
1,1.0,0.635165
2,1.0,0.766586
3,1.0,0.724564
4,1.0,0.889199


In [4]:
def confusion_matrix(df_a):

    TN = df_a.loc[(df_a['y'] == 0) & (df_a['proba']< 0.5)].count()[0]
    FN = df_a.loc[(df_a['y'] == 1) & (df_a['proba']< 0.5)].count()[0]

    FP = df_a.loc[(df_a['y'] == 0) & (df_a['proba']> 0.5)].count()[0]
    TP = df_a.loc[(df_a['y'] == 1) & (df_a['proba']> 0.5)].count()[0]

    return TN, FN, FP, TP

In [5]:
TN, FN, FP, TP = confusion_matrix(df_a)
con_matrix = np.array([[TN,FN],[FP,TP]])
print('confusion_matrix\n',con_matrix)

confusion_matrix
 [[    0     0]
 [  100 10000]]


In [6]:
def f1_score(df_a):
    TN, FN, FP, TP = confusion_matrix(df_a)
    Pr = TP/(TP+FP)
    Re = TP/(FN+TP)

    f1 = 2*Pr*Re/(Pr+Re)
    return f1


In [7]:
print('f1_score ',f1_score(df_a))

f1_score  0.9950248756218906


In [8]:
def auc_score(df_a):
    a_des = df_a.sort_values(by =['proba'], ascending=False)

    tpr = []
    fpr = []
    for t in range(0,len(a_des)):
        
        a_des['proba_t'] = np.where(a_des['proba']>=a_des.iloc[t]['proba'],1,0 )
        TN = len(a_des.loc[(a_des['y'] == 0) & (a_des['proba_t'] == 0)])
        FN = len(a_des.loc[(a_des['y'] == 1) & (a_des['proba_t']== 0)])

        FP = len(a_des.loc[(a_des['y'] == 0) & (a_des['proba_t']==1)])
        TP = len(a_des.loc[(a_des['y'] == 1) & (a_des['proba_t']== 1)])

        #print(confusion_matrix)
        tpr.append(TP/(TP+FN))
        fpr.append(FP/(TN+FP))
    AUC = np.trapz(np.array(tpr), np.array(fpr))
    return AUC

In [9]:
print("AUC score ",auc_score(df_a))

AUC score  0.48829900000000004


In [10]:
def accuracy_score(df_a):
    TN, FN, FP, TP = confusion_matrix(df_a)
    acc_score =((TP+TN)/(TN+FN+FP+TP))
    return acc_score

In [11]:
print("Accuracy ",accuracy_score(df_a))

Accuracy  0.9900990099009901




## B. Compute performance metrics for the given data

In [12]:
df_b=pd.read_csv(r'C:\Users\HP\OneDrive\Applied ai\Module 3\assignment\Performance_metrics_\5_b.csv')
df_b.head()

,y,proba
0,0,0.281035
1,0,0.465152
2,0,0.352793
3,0,0.157818
4,0,0.276648


In [13]:
TN, FN, FP, TP = confusion_matrix(df_b)
con_matrix = np.array([[TN,FN],[FP,TP]])
print('confusion_matrix\n', con_matrix)

confusion_matrix
 [[9761   45]
 [ 239   55]]


In [14]:
print('f1_score ',f1_score(df_b))

f1_score  0.2791878172588833


In [15]:
print("AUC score ",auc_score(df_b))

AUC score  0.9377570000000001


In [16]:
print("Accuracy ",accuracy_score(df_b))

Accuracy  0.9718811881188119


### C. Compute the best threshold (similarly to ROC curve computation) of probability which gives lowest values of metric

In [17]:
df_c=pd.read_csv(r'C:\Users\HP\OneDrive\Applied ai\Module 3\assignment\Performance_metrics_\5_c.csv')
df_c.head()

,y,prob
0,0,0.458521
1,0,0.505037
2,0,0.418652
3,0,0.412057
4,0,0.375579


In [18]:
metric_A = {}
c_des = df_c.sort_values(by=['prob'], ascending=False)
unique_probs = c_des['prob'].unique()
for t in unique_probs:
    c_des['ypred'] = np.where(c_des['prob']<t,0,1)
    FN = len(c_des.loc[(c_des['y'] == 1) & (c_des['ypred']== 0)])
    FP = len(c_des.loc[(c_des['y'] == 0) & (c_des['ypred']==1)])
    A = 500*FN + 100*FP
    metric_A[t] = A
#print(metric_A)
min_val = min(metric_A.values())
print(min_val)

min_val_key = [k for k in metric_A if metric_A[k] == min_val]
best_threshold = min_val_key[0]
print("best threshold ",best_threshold)


141000
best threshold  0.230039028



## D.</b></font> Compute performance metrics(for regression) for the given data

In [19]:
df_d=pd.read_csv(r'C:\Users\HP\OneDrive\Applied ai\Module 3\assignment\Performance_metrics_\5_d.csv')
df_d.head()

,y,pred
0,101.0,100.0
1,120.0,100.0
2,131.0,113.0
3,164.0,125.0
4,154.0,152.0


In [20]:
error = []
sstotal = []
se = []
y_mean = (df_d['y'].sum())/len(df_d)
for i in range(len(df_d)):
    err = (abs((df_d.iloc[i]['y']) - (df_d.iloc[i]['pred'])))
    ss = ((df_d.iloc[i]['y']) - y_mean)
    error.append(err)
    sstotal.append(ss*ss)
    se.append((err*err))

In [21]:
mape = (sum(error)/(df_d['y'].sum()))*100
mse = sum(se)/len(df_d)
rres = (1 - (sum(se)/sum(sstotal)))
print('mean absolute percentage error ',mape)
print('Mean squared error',mse )
print('R squared',rres)

mean absolute percentage error  12.91202994009687
Mean squared error 177.16569974554707
R squared 0.9563582786990964
